In [1]:
pip install yfinance



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install openpyxl


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Import libraries
import pandas as pd
import numpy as np
import yfinance as yf


In [4]:
# Load Election Data
election_data_file = r'C:\Users\bruch\Desktop\Election_Data_V3.xlsx'
election_data = pd.read_excel(election_data_file)
election_data['Date'] = pd.to_datetime(election_data['Date'], errors='coerce')


In [5]:
# Filter election data for the last 20 years
last_20_years = pd.to_datetime('today') - pd.DateOffset(years=20)
filtered_election_data = election_data[election_data['Date'] >= last_20_years]


In [6]:
# Load data - Stocks 
tickers = ['^FTSE', '^GDAXI', 'FTSEMIB.MI', '^AEX', '^FCHI', '^IBEX']
index_to_country = {'^FTSE': 'UK', '^GDAXI': 'Germany', 'FTSEMIB.MI': 
                    'Italy', '^AEX': 'Netherlands', '^FCHI': 'France', '^IBEX': 'Spain'}

start_date = '2020-01-01'
end_date = '2022-12-31'


dfs = []
for index in tickers:
    data = yf.download(index, start=start_date, end=end_date)
    data['Index'] = index
    dfs.append(data)
index_data = pd.concat(dfs)
index_data['Country'] = index_data['Index'].map(index_to_country)
index_data.head(10)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Index,Country
Date,,,,,,,,
2020-01-02,7542.399902,7624.799805,7542.399902,7604.299805,7604.299805,482206700,^FTSE,UK
2020-01-03,7604.299805,7626.399902,7551.000000,7622.399902,7622.399902,475628900,^FTSE,UK
2020-01-06,7622.399902,7622.399902,7528.100098,7575.299805,7575.299805,516783400,^FTSE,UK
2020-01-07,7575.299805,7604.600098,7562.100098,7573.899902,7573.899902,511576100,^FTSE,UK
2020-01-08,7573.899902,7579.500000,7525.000000,7574.899902,7574.899902,606225000,^FTSE,UK
2020-01-09,7574.899902,7627.799805,7574.899902,7598.100098,7598.100098,653838100,^FTSE,UK
2020-01-10,7598.100098,7632.000000,7585.100098,7587.899902,7587.899902,651315600,^FTSE,UK
2020-01-13,7587.899902,7635.799805,7582.500000,7617.600098,7617.600098,659019400,^FTSE,UK
2020-01-14,7617.600098,7650.700195,7588.799805,7622.399902,7622.399902,697648200,^FTSE,UK


In [ ]:
def calculate_optimal_investment_plan(df, election_df, window_before=182, window_after=182):
    ''' 
    Calculate the optimal investment plan (days before and after the election) for each country.

    Parameters:
    - df: DataFrame containing historical stock data with 'Close' prices and a 'Country' column.
    - election_df: DataFrame containing election information with 'Date' column.
    - window_before: Maximum number of days before the election to consider.
    - window_after: Maximum number of days after the election to consider.

    Returns:
    - results: A list of dictionaries containing information about the optimal investment time for each country.
      Each dictionary has keys: 'Country', 'Election_Date', 'Buy_Days_Before', 'Sell_Days_After', 'Profit_Percentage'.
    '''
    # Get unique countries from the stock DataFrame
    unique_countries = df['Country'].unique()

    # Initialize a list to store results for each country
    results = []

    for country in unique_countries:
        # Filter stock data for the current country
        country_data = df[df['Country'] == country]

        for _, election_row in election_df[election_df['Country'] == country].iterrows():
            # Election date for the current country
            election_date = election_row['Date']

            # Skip rows with 'NaT' (Not a Time) values
            if pd.isna(election_date):
                continue

            # Calculate start and end dates for the analysis window
            start_date = election_date - pd.DateOffset(days=window_before)
            end_date = election_date + pd.DateOffset(days=window_after)

            # Filter stock data for the analysis window
            stock_data = country_data[(country_data['Date'] >= start_date) & (country_data['Date'] <= end_date)]

            if not stock_data.empty:
                # Calculate profits using vectorized operations
                stock_data['Profit'] = (stock_data['Close'].shift(-window_after) - stock_data['Close']) / stock_data['Close']

                # Find the index of the maximum profit
                max_profit_index = stock_data['Profit'].idxmax()

                # Calculate profit percentage
                buy_price = stock_data.loc[election_date, 'Close']
                sell_price = stock_data.loc[max_profit_index, 'Close']
                profit_percentage = ((sell_price - buy_price) / buy_price) * 100

                result_dict = {
                    'Country': country,
                    'Election_Date': election_date,
                    'Buy_Days_Before': (election_date - stock_data.index[0]).days,
                    'Sell_Days_After': (max_profit_index - election_date).days,
                    'Profit_Percentage': profit_percentage
                }

                results.append(result_dict)

    return results


In [8]:
def calculate_max_profit(df, window_size=12):
    ''' 
    Calculate the maximum profit that can be made by buying and selling a stock.

    Parameters:
    - df: DataFrame containing historical stock data with 'Close' prices.
    - window_size: Number of days to consider for calculating the maximum profit.

    Returns:
    - max_profit: Maximum profit that can be made.
    - buy_date: Date to buy the stock for maximum profit.
    - sell_date: Date to sell the stock for maximum profit.
    '''
    max_profit = 0
    buy_date = None
    sell_date = None
    
    for i in range(len(df) - window_size):
        for j in range(i + window_size, len(df)):
            profit = (df['Close'].iloc[j] - df['Close'].iloc[i]) / df['Close'].iloc[i]
            if profit > max_profit:
                max_profit = profit
                buy_date = df.index[i]
                sell_date = df.index[j]

    return max_profit, buy_date, sell_date
